**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets. <br>
0 -> no hate speech <br>
1 -> contains hate speech <br>

**Evaluation metric**
macro f1 score

> **Load the `clean data` preprocessed in `Assignment 1`, then handle it to be used with `RNNs`**

### Import Libraries

### Load Dataset

### Preprocessing

### Modelling

#### Evaluation

### Enhancement

### Results & Conclusion

#### Done!